In [17]:
import weave
import new_api
import pandas as pd

In [5]:
client = weave.init_local_client()

In [12]:
import openai
@weave.op()
def predict(model_name, math, temp, trial):
    result = openai.chat.completions.create(model=model_name, messages=[{'role': 'user', 'content': math}])
    return result.choices[0].message.content

s = new_api.weave_sweep(predict,
                        # dataset
                        math=['1 + 2', '2 ** 64', 'x *** z'],
                        # model
                        model_name=['gpt-3.5-turbo', 'gpt-4o'],
                        temp=[0, 1],
                        # trials
                        trial=[0, 1])
print('Predicting')
results = s.compute()

Predicting
cache_hits: 24


0it [00:00, ?it/s]


In [22]:
maths = []
for c in client.calls():
    maths.append({'math': c.inputs['math']})
#maths = pd.json_normalize(maths)

In [24]:
type(maths[0])

dict

In [20]:
@weave.op()
def classify(math):
    return len(math)

new_api.weave_map(maths, classify)

cache_hits: 3


0it [00:00, ?it/s]


[({'math': '1 + 2'}, 5), ({'math': '2 ** 64'}, 7), ({'math': 'x *** z'}, 7)]

In [21]:
client.calls().add_columns(lambda c: {'len': classify(c.inputs['math'])})

AttributeError: 'CallsIter' object has no attribute 'add_columns'